In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras import optimizers
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [2]:
# NAME = "Q2_Adam_DenseDropOut"
# tensorboard = TensorBoard(log_dir="1dCNN-{}".format(NAME))

In [3]:
# import competition dataset
x_train = np.load('train_data.npy')
y_train = pd.read_csv('train_labels.csv')
y_train = np.array(y_train)
y_train = y_train[:,1]

x_test = np.load('test_data.npy')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalization    
#normalizer = Normalizer().fit(x_train)
#x_train = normalizer.transform(x_train)
#x_test = normalizer.transform(x_test)

# reshape
x_train = x_train.reshape(-1,x_train.shape[1],1)
x_test = x_test.reshape(-1,x_test.shape[1],1)


In [4]:
# make output a 2 dim vector indicating class
y_train = np_utils.to_categorical(y_train, 2)

In [5]:
# create 1-D CNN

model = Sequential()
model.add(Conv1D(32, kernel_size=5, 
                 activation='relu',
                 input_shape=(x_train.shape[1],1))) 
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=50,stride=10))

model.add(Conv1D(64, kernel_size=5, 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=50,stride=10))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


W0924 00:17:25.924477  8420 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0924 00:17:25.966012  8420 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0924 00:17:25.971917  8420 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0924 00:17:26.264611  8420 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please us

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 44096, 32)         192       
_________________________________________________________________
batch_normalization_1 (Batch (None, 44096, 32)         128       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4405, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4401, 64)          10304     
_________________________________________________________________
batch_normalization_2 (Batch (None, 4401, 64)          256       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 436, 64)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 27904)             0         
__________

In [7]:
# compile model

adam=optimizers.Adam(lr=.0001) 
model.compile(loss='binary_crossentropy', 
              optimizer=adam,
              metrics=['accuracy'])

W0924 00:17:27.234030  8420 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0924 00:17:27.247353  8420 deprecation_wrapper.py:119] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0924 00:17:27.257280  8420 deprecation.py:323] From C:\Users\willi\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
from keras.callbacks import EarlyStopping
EarlyStopping = EarlyStopping(monitor='val_loss',patience=5,verbose=0,mode='auto')
model.fit(x_train, y_train, batch_size=32, epochs=30, validation_split=0.1,callbacks=[EarlyStopping]) #callbacks=[tensorboard]


Train on 3599 samples, validate on 400 samples
Epoch 1/30
3599/3599 [==============================] - 610s 170ms/step - loss: 0.6055 - acc: 0.9130 - val_loss: 0.2024 - val_acc: 0.9525
Epoch 2/30
3599/3599 [==============================] - 747s 208ms/step - loss: 0.3458 - acc: 0.9433 - val_loss: 0.1336 - val_acc: 0.9725
Epoch 3/30
3599/3599 [==============================] - 602s 167ms/step - loss: 0.2231 - acc: 0.9572 - val_loss: 0.1389 - val_acc: 0.9700
Epoch 4/30
3599/3599 [==============================] - 34423s 10s/step - loss: 0.1866 - acc: 0.9600 - val_loss: 0.1083 - val_acc: 0.9750
Epoch 5/30
3599/3599 [==============================] - 613s 170ms/step - loss: 0.1595 - acc: 0.9650 - val_loss: 0.1292 - val_acc: 0.9650
Epoch 6/30
3599/3599 [==============================] - 654s 182ms/step - loss: 0.1688 - acc: 0.9628 - val_loss: 0.1246 - val_acc: 0.9800
Epoch 7/30
3599/3599 [==============================] - 640s 178ms/step - loss: 0.1314 - acc: 0.9722 - val_loss: 0.1141 - val

In [12]:
# make predictions
y_predict= model.predict_classes(x_test)
print(y_predict[:500])

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0
 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [14]:
prediction = pd.DataFrame(y_predict,columns=['label']).to_csv('submission_new.csv')

In [ ]:
'''
TrainingNotes:
# original //0.0583 - acc: 0.9825 - val_loss: 0.1250 - val_acc: 0.9650
# add batchNormalization and dropout(0.5) on all conv layer and validation split=0.1 // not good
* only add batchNormalization on conv //loss: 0.0962 - acc: 0.9797 - val_loss: 0.1720 - val_acc: 0.9725
# 2nd convLayer from 64 to 128.and maxpooling 50 to 60 //loss: 0.2091 - acc: 0.9644 - val_loss: 0.1568 - val_acc: 0.9675
'''